In [6]:
import os 
import sys
import numpy as np 
import torch

import torchaudio

from tqdm.notebook import tqdm

/home/ankita/anaconda3/envs/audio1/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
import whisper

modelw = whisper.load_model("base.en")
print(
    f"Model is {'multilingual' if modelw.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in modelw.parameters()):,} parameters."
)

Model is English-only and has 71,825,408 parameters.


In [26]:
options = whisper.DecodingOptions(language="en", without_timestamps=True)

In [5]:
import pickle
model = pickle.load(open('bird_vs_back.m', "rb"))

In [28]:
device = "cuda" if torch.cuda.is_available() else "cpu"


In [14]:
#  Location of the folder where the .wav files are saved
flds = os.listdir('Folder_name/')
# flds = os.listdir('/home/ankita/Code/XprizeZoo/XPRIZE/LL_Yellow/Audio/WA_202306040930_nn/')


for fl in flds:
   
    audio, sr = torchaudio.load('Folder_name/{}'.format(fl))

    audio1 = whisper.pad_or_trim(audio.flatten()).to(device)
    mel = whisper.log_mel_spectrogram(audio1)

    results = modelw.decode(mel.to(device), options)

    d = np.array(results.audio_features.cpu().numpy())
    feat_te= (np.mean(d, axis=0))[np.newaxis,:]

    pred = model.predict(feat_te)
    # print(pred)
    if pred ==0:

        with open('birds_detected_on_ground.txt', 'a') as f:
            f.write(fl +'\n')
            # f.write('\n')

SMM10035-Y_20230603_223902.wav
SMM10035-Y_20230603_235902.wav
SMM10035-Y_20230603_222102.wav
SMM10035-Y_20230603_215702.wav
SMM10035-Y_20230603_170002.wav
SMM10035-Y_20230603_182602.wav
SMM10035-Y_20230603_201902.wav
SMM10035-Y_20230603_211602.wav
SMM10035-Y_20230603_201602.wav
SMM10035-Y_20230603_202802.wav
SMM10035-Y_20230603_235302.wav
SMM10035-Y_20230603_180702.wav
SMM10035-Y_20230603_212302.wav
SMM10035-Y_20230603_173302.wav
SMM10035-Y_20230603_221102.wav
SMM10035-Y_20230603_150502.wav
SMM10035-Y_20230603_184502.wav
SMM10035-Y_20230603_191302.wav
SMM10035-Y_20230603_235202.wav
SMM10035-Y_20230603_180202.wav
SMM10035-Y_20230603_163002.wav
SMM10035-Y_20230603_191702.wav
SMM10035-Y_20230603_214202.wav
SMM10035-Y_20230603_232502.wav
SMM10035-Y_20230603_181402.wav
SMM10035-Y_20230603_210602.wav
SMM10035-Y_20230603_180502.wav
SMM10035-Y_20230603_160602.wav
SMM10035-Y_20230603_224002.wav
SMM10035-Y_20230603_190902.wav
SMM10035-Y_20230603_234602.wav
SMM10035-Y_20230603_185702.wav
SMM10035

In [3]:
import soundfile as sf
import librosa.display
# Get number of samples for 2 seconds; replace 2 by any number
with open('birds_detected_on_ground.txt', 'r+') as f:



    for line in f.readlines():
        fl = line.split('\n')[0]

        # out_filename = 'aaaa'
        loc = 'Folder_name/'+fl

        #  chop the .wav files

        audio, sr= librosa.load(loc)
        buffer = 10 * sr

        samples_total = len(audio)
        samples_wrote = 0
        counter = 1

        while samples_wrote < samples_total:

            #check if the buffer is not exceeding total samples 
            if buffer > (samples_total - samples_wrote):
                buffer = samples_total - samples_wrote

            block = audio[samples_wrote : (samples_wrote + buffer)]
            # out_filename = fl +"split_" + str(counter) + "_"+".wav"
            out_filename = "split_" + str(counter) + "_" +fl


            # Write 2 second segment
            sf.write('New_folder_name/'+out_filename, block, sr)
            counter += 1
            samples_wrote += buffer

In [23]:
import pickle
model_birds= pickle.load(open('ID_bird.m', "rb"))

In [ ]:

flds = os.listdir('New_folder_name/')



for fl in flds:

   
    audio, sr = torchaudio.load('New_folder_name/{}'.format(fl))

    # audio1, sample_rate2 = torchaudio.sox_effects.apply_effects_tensor(audio, sr, effects)
    audio1 = whisper.pad_or_trim(audio.flatten()).to(device)
    mel = whisper.log_mel_spectrogram(audio1)

    results = modelw.decode(mel.to(device), options)

    d = np.array(results.audio_features.cpu().numpy())
    feat_te= (np.mean(d, axis=0))[np.newaxis,:]



    pred = np.around(model_birds.predict_proba(feat_te), 3)
    data_in = str(pred.tolist())[2:-2]
    data_in = data_in.replace(',', '  ')

    with open('bird_predicted.txt', 'a') as f:

        f.write(fl + ' '+data_in+'\n')
